<a href="https://colab.research.google.com/github/rposhala/En-Fr_Translation-Toolbar_with_AutoCorrect-AutoFill/blob/AutoCorrect/AutoCorrect.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import re
from collections import Counter
import numpy as np
import pandas as pd

In [5]:
def process_data(file_name):
    words = []
    with open(file_name) as f:
        read_file = f.read()
        for line in read_file.lower().split('\n'):
            words += re.findall(r'\w+', line)
    
    return words

In [6]:
# I used my google drive manage the dataset
import os
from google.colab import drive
drive.mount('/content/drive')
os.chdir("drive/MyDrive/Colab Notebooks/NLP_Specialization")
!ls

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
AutoCorrect.ipynb	fr_embeddings.p
Course2-NLPProbModels	GoogleNews-vectors-negative300.bin
en_embeddings.p		GoogleNews-vectors-negative300.bin.gz
en-fr.test.txt		NLP_with_Classification_and_Vector_Spaces
en-fr.train.txt		shakespeare.txt
En-Fr-Translator.ipynb	wiki.multi.fr.vec


In [8]:
word_l = process_data('./shakespeare.txt')
vocab = set(word_l)
print("The first ten words in the text are: ", word_l[0:10])
print(f"There are {len(vocab)} unique words in the vocabulary.")

The first ten words in the text are:  ['o', 'for', 'a', 'muse', 'of', 'fire', 'that', 'would', 'ascend', 'the']
There are 6116 unique words in the vocabulary.


In [10]:
def get_count(word_l):
    word_count_dict = {}
    word_count_dict = Counter(word_l)
    return word_count_dict

In [12]:
word_count_dict = get_count(word_l)
print("There are ",len(word_count_dict)," key values pairs")
print("The count for the word 'thee' is ",word_count_dict.get('thee',0))

There are  6116  key values pairs
The count for the word 'thee' is  240


In [13]:
def get_probs(word_count_dict):
    probs = {}    
    total = sum([i for i in word_count_dict.values()])
    for k, v in word_count_dict.items():
        probs[k] = v/total

    return probs

In [14]:
probs = get_probs(word_count_dict)
print(f"Length of probs is {len(probs)}")
print(f"P('thee') is {probs['thee']:.4f}")

Length of probs is 6116
P('thee') is 0.0045


<a name='2'></a>
## String Manipulations

Now, that you have computed $P(w_i)$ for all the words in the corpus, you will write a few functions to manipulate strings so that you can edit the erroneous strings and return the right spellings of the words. In this section, you will implement four functions: 

* delete_letter: given a word, it returns all the possible strings that have **one character removed**. 
* switch_letter: given a word, it returns all the possible strings that have **two adjacent letters switched**.
* replace_letter: given a word, it returns all the possible strings that have **one character replaced by another different letter**.
* insert_letter: given a word, it returns all the possible strings that have an **additional character inserted**. 


In [15]:
def delete_letter(word, verbose=False):

    delete_l = []
    split_l = []

    split_l = [(word[:i], word[i:]) for i in range(len(word))]
    delete_l = [s1+s2[1:] for s1, s2 in split_l]

    if verbose: print(f"input word {word}, \nsplit_l = {split_l}, \ndelete_l = {delete_l}")

    return delete_l

In [16]:
delete_word_l = delete_letter(word="cans",
                        verbose=True)

input word cans, 
split_l = [('', 'cans'), ('c', 'ans'), ('ca', 'ns'), ('can', 's')], 
delete_l = ['ans', 'cns', 'cas', 'can']


In [17]:
def switch_letter(word, verbose=False):
    
    switch_l = []
    split_l = []
    
    split_l = [(word[:i], word[i:]) for i in range(len(word))]
    switch_l = [s1+s2[1]+s2[0]+s2[2:] for s1, s2 in split_l if len(s2) > 1]
    
    if verbose: print(f"Input word = {word} \nsplit_l = {split_l} \nswitch_l = {switch_l}") 

    return switch_l

In [18]:
switch_word_l = switch_letter(word="eta", verbose=True)

Input word = eta 
split_l = [('', 'eta'), ('e', 'ta'), ('et', 'a')] 
switch_l = ['tea', 'eat']


In [19]:
def replace_letter(word, verbose=False):
    
    letters = 'abcdefghijklmnopqrstuvwxyz'
    replace_l = []
    split_l = []
    
    split_l = [(word[:i], word[i:]) for i in range(len(word))]
    replace_set = set(s1+i+s2[1:] for s1, s2 in split_l for i in letters if s1+i+s2[1:] != word)
    
    replace_l = sorted(list(replace_set))
    
    if verbose: print(f"Input word = {word} \nsplit_l = {split_l} \nreplace_l {replace_l}")   
    
    return replace_l

In [20]:
replace_l = replace_letter(word='can',
                              verbose=True)

Input word = can 
split_l = [('', 'can'), ('c', 'an'), ('ca', 'n')] 
replace_l ['aan', 'ban', 'caa', 'cab', 'cac', 'cad', 'cae', 'caf', 'cag', 'cah', 'cai', 'caj', 'cak', 'cal', 'cam', 'cao', 'cap', 'caq', 'car', 'cas', 'cat', 'cau', 'cav', 'caw', 'cax', 'cay', 'caz', 'cbn', 'ccn', 'cdn', 'cen', 'cfn', 'cgn', 'chn', 'cin', 'cjn', 'ckn', 'cln', 'cmn', 'cnn', 'con', 'cpn', 'cqn', 'crn', 'csn', 'ctn', 'cun', 'cvn', 'cwn', 'cxn', 'cyn', 'czn', 'dan', 'ean', 'fan', 'gan', 'han', 'ian', 'jan', 'kan', 'lan', 'man', 'nan', 'oan', 'pan', 'qan', 'ran', 'san', 'tan', 'uan', 'van', 'wan', 'xan', 'yan', 'zan']


In [21]:
def insert_letter(word, verbose=False):
    letters = 'abcdefghijklmnopqrstuvwxyz'
    insert_l = []
    split_l = []
    
    split_l = [(word[:i], word[i:]) for i in range(len(word)+1)]
    insert_l = [s1+i+s2 for s1, s2 in split_l for i in letters]
    
    if verbose: print(f"Input word {word} \nsplit_l = {split_l} \ninsert_l = {insert_l}")
    
    return insert_l

In [22]:
insert_l = insert_letter('at', True)
print(f"Number of strings output by insert_letter('at') is {len(insert_l)}")

Input word at 
split_l = [('', 'at'), ('a', 't'), ('at', '')] 
insert_l = ['aat', 'bat', 'cat', 'dat', 'eat', 'fat', 'gat', 'hat', 'iat', 'jat', 'kat', 'lat', 'mat', 'nat', 'oat', 'pat', 'qat', 'rat', 'sat', 'tat', 'uat', 'vat', 'wat', 'xat', 'yat', 'zat', 'aat', 'abt', 'act', 'adt', 'aet', 'aft', 'agt', 'aht', 'ait', 'ajt', 'akt', 'alt', 'amt', 'ant', 'aot', 'apt', 'aqt', 'art', 'ast', 'att', 'aut', 'avt', 'awt', 'axt', 'ayt', 'azt', 'ata', 'atb', 'atc', 'atd', 'ate', 'atf', 'atg', 'ath', 'ati', 'atj', 'atk', 'atl', 'atm', 'atn', 'ato', 'atp', 'atq', 'atr', 'ats', 'att', 'atu', 'atv', 'atw', 'atx', 'aty', 'atz']
Number of strings output by insert_letter('at') is 78


In [23]:
def edit_one_letter(word, allow_switches = True):
    
    edit_one_set = set()
    
    edit_one_set = edit_one_set.union(set(delete_letter(word)+replace_letter(word)+insert_letter(word)))
    if allow_switches:
        edit_one_set = edit_one_set.union(set(switch_letter(word)))
    
    return edit_one_set

In [24]:
tmp_word = "at"
tmp_edit_one_set = edit_one_letter(tmp_word)
tmp_edit_one_l = sorted(list(tmp_edit_one_set))

print(f"input word {tmp_word} \nedit_one_l \n{tmp_edit_one_l}\n")

input word at 
edit_one_l 
['a', 'aa', 'aat', 'ab', 'abt', 'ac', 'act', 'ad', 'adt', 'ae', 'aet', 'af', 'aft', 'ag', 'agt', 'ah', 'aht', 'ai', 'ait', 'aj', 'ajt', 'ak', 'akt', 'al', 'alt', 'am', 'amt', 'an', 'ant', 'ao', 'aot', 'ap', 'apt', 'aq', 'aqt', 'ar', 'art', 'as', 'ast', 'ata', 'atb', 'atc', 'atd', 'ate', 'atf', 'atg', 'ath', 'ati', 'atj', 'atk', 'atl', 'atm', 'atn', 'ato', 'atp', 'atq', 'atr', 'ats', 'att', 'atu', 'atv', 'atw', 'atx', 'aty', 'atz', 'au', 'aut', 'av', 'avt', 'aw', 'awt', 'ax', 'axt', 'ay', 'ayt', 'az', 'azt', 'bat', 'bt', 'cat', 'ct', 'dat', 'dt', 'eat', 'et', 'fat', 'ft', 'gat', 'gt', 'hat', 'ht', 'iat', 'it', 'jat', 'jt', 'kat', 'kt', 'lat', 'lt', 'mat', 'mt', 'nat', 'nt', 'oat', 'ot', 'pat', 'pt', 'qat', 'qt', 'rat', 'rt', 'sat', 'st', 't', 'ta', 'tat', 'tt', 'uat', 'ut', 'vat', 'vt', 'wat', 'wt', 'xat', 'xt', 'yat', 'yt', 'zat', 'zt']



In [25]:
def edit_two_letters(word, allow_switches = True):
    
    edit_two_set = set()
    
    edit_one_set = edit_one_letter(word, allow_switches)
    one_edit_list = [edit_one_letter(edited_w, allow_switches) for edited_w in edit_one_set]
    for i in one_edit_list:
        edit_two_set = edit_two_set.union(i)
    
    return edit_two_set

In [26]:
tmp_edit_two_set = edit_two_letters("a")
tmp_edit_two_l = sorted(list(tmp_edit_two_set))
print(f"Number of strings with edit distance of two: {len(tmp_edit_two_l)}")
print(f"First 10 strings {tmp_edit_two_l[:10]}")
print(f"Last 10 strings {tmp_edit_two_l[-10:]}")
print(f"The data type of the returned object should be a set {type(tmp_edit_two_set)}")
print(f"Number of strings that are 2 edit distances from 'at' is {len(edit_two_letters('at'))}")

Number of strings with edit distance of two: 2654
First 10 strings ['', 'a', 'aa', 'aaa', 'aab', 'aac', 'aad', 'aae', 'aaf', 'aag']
Last 10 strings ['zv', 'zva', 'zw', 'zwa', 'zx', 'zxa', 'zy', 'zya', 'zz', 'zza']
The data type of the returned object should be a set <class 'set'>
Number of strings that are 2 edit distances from 'at' is 7154


In [27]:
def get_corrections(word, probs, vocab, n=2, verbose = False):

    suggestions = []
    n_best = []
    suggestions = list((word in vocab and word) or edit_one_letter(word).intersection(vocab) or edit_two_letters(word).intersection(vocab) or word)
    n_best = sorted([(i, probs[i]) for i in list(suggestions)], key = lambda x: -x[1])[:n]
    
    if verbose: print("entered word = ", word, "\nsuggestions = ", suggestions)

    return n_best

In [28]:
my_word = 'dys' 
tmp_corrections = get_corrections(my_word, probs, vocab, 2, verbose=True) # keep verbose=True
for i, word_prob in enumerate(tmp_corrections):
    print(f"word {i}: {word_prob[0]}, probability {word_prob[1]:.6f}")

print(f"data type of corrections {type(tmp_corrections)}")

entered word =  dys 
suggestions =  ['days', 'dye']
word 0: days, probability 0.000410
word 1: dye, probability 0.000019
data type of corrections <class 'list'>


## Minimum Edit distance

We need to evaluate the similarity between two strings like 'waht' and 'what' to implement auto-correct with the functionality pieces we coded.
And we also need to find an efficient shortest path to go from the word, 'waht' to the word 'what'

We need to implement a dynamic programming system that will tell you the minimum number of edits required to convert a string into another string.

In [29]:
def min_edit_distance(source, target, ins_cost = 1, del_cost = 1, rep_cost = 2):
    
    
    m = len(source) 
    n = len(target) 
    #initialize cost matrix with zeros and dimensions (m+1,n+1) 
    D = np.zeros((m+1, n+1), dtype=int) 
    
    for row in range(1,m+1):
        D[row,0] = row
        
    for col in range(1,n+1):
        D[0,col] = col
        
    for row in range(1,m+1): 
        for col in range(1,n+1):
            
            r_cost = rep_cost
            if source[row-1] == target[col-1]:
                r_cost = 0
            D[row,col] = min(D[row-1, col]+ins_cost, D[row][col-1]+del_cost, D[row-1][col-1]+r_cost)
          
    # Set the minimum edit distance with the cost found at row m, column n
    med = D[m][n]
    
    return D, med

In [30]:
source =  'play'
target = 'stay'
matrix, min_edits = min_edit_distance(source, target)
print("minimum edits: ",min_edits, "\n")
idx = list('#' + source)
cols = list('#' + target)
df = pd.DataFrame(matrix, index=idx, columns= cols)
print(df)

minimum edits:  4 

   #  s  t  a  y
#  0  1  2  3  4
p  1  2  3  4  5
l  2  3  4  5  6
a  3  4  5  4  5
y  4  5  6  5  4


In [31]:
source =  'eer'
target = 'near'
matrix, min_edits = min_edit_distance(source, target)
print("minimum edits: ",min_edits, "\n")
idx = list(source)
idx.insert(0, '#')
cols = list(target)
cols.insert(0, '#')
df = pd.DataFrame(matrix, index=idx, columns= cols)
print(df)

minimum edits:  3 

   #  n  e  a  r
#  0  1  2  3  4
e  1  2  1  2  3
e  2  3  2  3  4
r  3  4  3  4  3


## Need to implement backtracking algorithm